# Проверка работы функций из src.utills и src.recommender

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from implicit.nearest_neighbours import ItemItemRecommender 

In [11]:
# импорт модулей src.utils и src.recommenders

from src.utils import precision_at_k, prefilter_items, postfilter_items, popular_items, gen_dicts
from src.utils import get_similar_items_recommendation, get_similar_users_recommendation

###  Проверка работы get_similar_items_recommendation
Берем топ-5 популярных покупок юзера --> находим по 1 наиболее близкому товару для них --> рекомендуем

In [3]:
# читаем данные из файла
data = pd.read_csv('transaction_data.csv')

# приводим названия столбцов к нижнему регистру и переименовываем два столбца
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3
# делим выборку на train и test
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# в тестовых данных для каждого пользователя находим список купленных товаров
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

# в обучающей выборке берем только 5000 самых популярных товаров
data_train = popular_items(data=data_train)

In [4]:
# строим матрицу c_ui
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
c_ui = user_item_matrix 
user_item_matrix = bm25_weight(user_item_matrix.T).T  # применям к c_ui матрице bm25 веса 

In [5]:
%%time

# обучаем ALS модель на матрице c_ui
model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                              show_progress=True)


Wall time: 1.18 s


In [6]:
# делаем предсказание с помощью построенной модели для каждого пользователя
res = get_similar_items_recommendation(model, data_train, c_ui, N=5)

In [7]:
res.head(6)

,user_id,similar_items_rec
0,1,"[980644, 824758, 1132231, 940108, 864774]"
1,2,"[850529, 960318, 1133018, 880888, 819978]"
2,3,"[1075979, 960318, 1096727, 1076875, 901543]"
3,4,"[1074754, 1084310, 945998, 1105301, 843450]"
4,5,"[1058997, 946839, 833025, 12262778, 1105301]"
5,6,"[948650, 5569845, 889731, 934639, 1037863]"


###  Проверка работы get_similar_users_recommendation

Пользователь --> похожие пользователи --> рекомендовать те товары, которые купили похожие пользователи

In [8]:
userids = c_ui.index.values
itemids = c_ui.columns.values

id_to_userid, userid_to_id = gen_dicts(userids)
id_to_itemid, itemid_to_id = gen_dicts(itemids)

In [9]:
%%time

own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

own.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)



Wall time: 427 ms


In [13]:
%%time
# Ждем исполнения минут 30
#res1 = get_similar_users_recommendation(own, data_train, c_ui, N=5)

Wall time: 0 ns


In [118]:
%%time

def get_rec(users, model, N=2):
    res = []
    for user in users:
        try:
            res += [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(c_ui).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # фильтруем 999999
                                    recalculate_user=True)]
        except KeyError:
            res = []
    res = list(set(res))[:5]
    return res



result['rec_item'] = result['similar_users'].apply(lambda x: get_rec(x, model=own))

Wall time: 40min


In [119]:
result

,user_id,item_id,similar_users,rec_item
0,1,1625,"[1899, 711, 679, 2373, 207]","[6533889, 1138189, 879504, 1037840, 1404121]"
1,2,714,"[2196, 1844, 2384, 2207, 1734]","[999104, 6533889, 819840, 885290, 1058939]"
2,3,902,"[296, 2079, 1764, 1547, 2169]","[1104224, 6533889, 6534178, 5565925, 13511722]"
3,4,301,"[1781, 661, 2382, 1826, 398]","[6533889, 1028816, 1025650, 1070803, 1138292]"
4,5,219,"[282, 22, 447, 2459, 277]","[6533889, 942088, 839849, 1056651, 1057260]"
...,...,...,...,...
2495,2496,1489,"[1064, 2384, 1777, 2207, 2047]","[6533889, 885290, 862349, 1126899, 915349]"
2496,2497,1940,"[2227, 160, 2483, 863, 399]","[6533889, 846241, 1075368, 917033, 5569135]"
2497,2498,843,"[792, 1888, 1072, 1233, 2274]","[6533889, 1038985, 907662, 834833, 866227]"
2498,2499,1050,"[1315, 1633, 2344, 1795, 775]","[1135681, 6533889, 890536, 1075827, 1070803]"
